# Imports and Setup

## Imports

In [ ]:
import rp
import torch
import einops
import numpy

from source.scene_reader import extract_seg_photo_rgb_and_labels, extract_scene_uvs_and_scene_labels
from source.projector import colorized_scene_labels
from source.unprojector import get_label_averge_colors

## Misc

In [ ]:
#Make the pixels of Jupyter-displayed images 
# use nearest-neigbor interpolation
from IPython.core.display import display, HTML
display(HTML("""
<style>
img {
  image-rendering: auto;
  image-rendering: crisp-edges;
  image-rendering: pixelated;
}
</style>
"""))

In [ ]:
def display_colors(colors):
    #Assumes 0<colors<1
    import numpy as np
    tiles = []
    for index,color in enumerate(colors):
        color = rp.as_numpy_array(color)
        tile = np.ones((128,128,3))
        tile = tile * color[None,None]
        tiles.append(tile)
    rp.display_image(rp.tiled_images(tiles))

## Settings

In [ ]:
seg_folder = 'datasets/five_items/photos_seg'
assert rp.folder_exists(seg_folder)

uvl_scene_folder = 'datasets/five_items/scenes'
assert rp.folder_exists(uvl_scene_folder)

label_values = [0,50,100,150,200,255]

In [ ]:
seg_images = rp.load_images(seg_folder, use_cache=True, show_progress=True)

# Seg Images

In [ ]:
seg_image = rp.random_element(seg_images)
alpha = seg_image[:,:,3]

rp.display_image(
    rp.horizontally_concatenated_images(
        seg_image,
        alpha,
    )
)

In [ ]:
seg_images = [rp.as_float_image(x) for x in seg_images]
torch_seg_images = rp.as_torch_images(rp.as_numpy_array(seg_images))

In [ ]:
rgb, labels = extract_seg_photo_rgb_and_labels(torch_seg_images, label_values)

In [ ]:
#Random colors we use to visualize the labels

colors = [
    [0.5 , 0   , 0.5 ] ,
    [0.5 , 1   , 0   ] ,
    [0   , 0.5 , 0   ] ,
    [0.5 , 0.5 , 0   ] ,
    [0   , 1   , 1   ] ,
    [0   , 0   , 1   ] ,
    [0   , 0.5 , 0.5 ] ,
    [0.5 , 1   , 0.5 ] ,
    [0   , 0   , 0   ] ,
    [0.5 , 0   , 0   ] ,
    [1   , 0   , 0.5 ] ,
    [0   , 0.5 , 1   ] ,
    [1   , 0.5 , 0   ] ,
    [1   , 1   , 1   ] ,
    [0   , 1   , 0   ] ,
    [0.5 , 0.5 , 0.5 ] ,
    [0.5 , 0.5 , 1   ] ,
    [1   , 1   , 0   ] ,
    [0.5 , 1   , 1   ] ,
    [0   , 0   , 0.5 ] ,
    [1   , 0   , 0   ] ,
    [0   , 1   , 0.5 ] ,
    [0.5 , 0   , 1   ] ,
    [1   , 0.5 , 1   ] ,
    [1   , 0   , 1   ] ,
    [1   , 1   , 0.5 ] ,
    [1   , 0.5 , 0.5 ] ,
]

colors = torch.tensor(rp.as_numpy_array(colors))

In [ ]:
uvl_scenes = rp.get_all_files(uvl_scene_folder)
uvl_scenes = rp.random_batch(uvl_scenes, 5)
uvl_scenes = rp.load_images (uvl_scenes   )
uvl_scenes = [rp.as_rgb_image  (x) for x in uvl_scenes]
uvl_scenes = [rp.as_float_image(x) for x in uvl_scenes]
uvl_scenes = rp.as_numpy_array (uvl_scenes)
uvl_scenes = rp.as_torch_images(uvl_scenes)

_, uvl_labels = extract_scene_uvs_and_scene_labels(uvl_scenes, label_values)

In [ ]:
index = rp.random_index(5)

print("Make sure all the labels in the photo match all the labels in the UVL scene!")
print("Its a good oppurtunity to double-check your annotations")

rp.display_image(
    rp.vertically_concatenated_images(
        rp.horizontally_concatenated_images(
            rp.labeled_image(
                rp.as_numpy_image(uvl_scenes[index]),
                'Synthetic UVL Scene',
                size=30,
            ),
            rp.labeled_image(
                rp.as_numpy_image(colorized_scene_labels(uvl_labels,colors)[index]),
                'Synthetic UVL Labels',
                size=30,
            ),
        ),
        rp.horizontally_concatenated_images(
            rp.labeled_image(
                rp.as_numpy_image(rgb[index]),
                'Photo',
                size=30,
            ),
            rp.labeled_image(
                rp.as_numpy_image(colorized_scene_labels(labels,colors)[index]),
                'Photo Labels',
                size=30,
            ),
        ),
    ),
)

# Average Colors Test

In [ ]:
colors = get_label_averge_colors(rgb, labels, len(label_values))
display_colors(colors)
print(rp.as_numpy_array(colors))

In [ ]:
index = rp.random_index(5)

print("Make sure all the labels in the photo match all the labels in the UVL scene!")
print("Its a good oppurtunity to double-check your annotations")

rp.display_image(
    rp.vertically_concatenated_images(
        rp.horizontally_concatenated_images(
            rp.labeled_image(
                rp.as_numpy_image(uvl_scenes[index]),
                'Synthetic UVL Scene',
                size=30,
            ),
            rp.labeled_image(
                rp.as_numpy_image(colorized_scene_labels(uvl_labels,colors)[index]),
                'Synthetic UVL Labels',
                size=30,
            ),
        ),
        rp.horizontally_concatenated_images(
            rp.labeled_image(
                rp.as_numpy_image(rgb[index]),
                'Photo',
                size=30,
            ),
            rp.labeled_image(
                rp.as_numpy_image(colorized_scene_labels(labels,colors)[index]),
                'Photo Labels',
                size=30,
            ),
        ),
    ),
)